In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
import sys
import numpy as np
import pandas as pd
import random
import collections

In [16]:
import re

In [33]:
from nltk.tokenize import word_tokenize

## Load Data

In [12]:
data_path = '../../../Data/'

In [14]:
data = pd.read_excel(data_path+'한국어_대화체_번역.xlsx' , engine='openpyxl')

In [17]:
data[['원문' , '번역문']].head()

,원문,번역문
0,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...


In [19]:
kor_data = data['원문']
en_data = data['번역문']

In [84]:
class VocabSet :
    
    def __init__(self, th=3) :
        
        self.th = th
        
    def tokens(self, data) :
        
        vocab_set = collections.Counter()
        
        for i , sen in enumerate(data) :
            tokens = word_tokenize(sen.lower())
            vocab_set.update(tokens)
                
        vocab_dict = dict(vocab_set)
        valid_tok = []
        
        for tok in vocab_dict.keys() :
            count = vocab_dict[tok]
            tok = tok
            
            if (count > self.th) and tok.isalpha() :
                valid_tok.append(tok)
                
        random.shuffle(valid_tok)
        valid_tok.append('utk')
                
        return dict(zip(valid_tok , range(1,len(valid_tok)+1)))
        

In [94]:
class Encoder :
    
    def __init__(self, data, token_dict) :
        
        self.data = data
        self.token_dict = token_dict
        
        self.sos = 0
        self.eos = len(token_dict) + 1
        
    def check(self, tok) :
        if tok in self.token_dict :
            return True
        else :
            return False
        
    def encode_sen(self, sen) :
        tokens = word_tokenize(sen)
        encoded = []
        
        for tok in tokens :
            if self.check(tok) :
                idx = self.token_dict[tok]
            else :
                idx = self.token_dict['utk']
            encoded.append(idx)
            
        encoded = [self.sos] + encoded + [self.eos]
            
        return encoded
    
    def encode(self) :
        encoded = []
        
        for i , sen in enumerate(self.data) :
            encoded_sen = self.encode_sen(sen)
            encoded.append(encoded_sen)
        
        return encoded